读取txt

In [122]:
import pandas as pd
data_path='old_train.csv'
test_path='old_test.csv'
columns = ['user_id', 'item_id']
df=pd.read_csv(data_path)
test_df=pd.read_csv(test_path)
print(df.head())

   user_id  item_id
0        0        0
1        0        1
2        0        2
3        0        3
4        0        4


In [123]:
import numpy as np
RANDOM_SEED = 0
NEG_ITEMS = 100

In [124]:
np.random.seed(RANDOM_SEED)

In [125]:
# out_df = df.rename(columns={'location_id': 'item_id', 'user_id': 'user_id', 'time_numeric': 'time'})
# out_df = out_df[['user_id', 'item_id', 'time']]
# out_df = out_df.drop_duplicates(['user_id', 'item_id', 'time'])
# out_df = out_df.sort_values(by=['time', 'user_id'], kind='mergesort').reset_index(drop=True)
# l_u=int(len(out_df['user_id'].unique())/6)
# l_i=int(len(out_df['item_id'].unique())/6)

# np.random.seed(42)
# sample_users=np.random.choice(out_df['user_id'].unique(),l_u,replace=False)
# sample_items=np.random.choice(out_df['item_id'].unique(),l_i,replace=False)

# filtered_df=out_df[out_df['user_id'].isin(sample_users) & out_df['item_id'].isin(sample_items)]

out_df=df
out_df.head()

,user_id,item_id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [126]:
# uids = sorted(out_df['user_id'].unique())
# user2id = dict(zip(uids, range(1, len(uids) + 1)))
# iids = sorted(out_df['item_id'].unique())
# item2id = dict(zip(iids, range(1, len(iids) + 1)))

# out_df['user_id'] = out_df['user_id'].apply(lambda x: user2id[x])
# out_df['item_id'] = out_df['item_id'].apply(lambda x: item2id[x])
# out_df.head()

In [127]:
# leave one out spliting

clicked_item_set = dict()
for user_id, seq_df in out_df.groupby('user_id'):
    clicked_item_set[user_id] = set(seq_df['item_id'].values.tolist())
    
def generate_dev_test(data_df):
    result_dfs = []
    n_items = data_df['item_id'].value_counts().size
    for idx in range(1):
        result_df = data_df.groupby('user_id').tail(1).copy()
        data_df = data_df.drop(result_df.index)
        neg_items = np.random.randint(1, n_items + 1, (len(result_df), NEG_ITEMS))
        for i, uid in enumerate(result_df['user_id'].values):
            user_clicked = clicked_item_set[uid]
            for j in range(len(neg_items[i])):
                while neg_items[i][j] in user_clicked:
                    neg_items[i][j] = np.random.randint(1, n_items + 1)
        result_df['neg_items'] = neg_items.tolist()
        result_dfs.append(result_df)
    return result_dfs, data_df

In [128]:
def generate_test_neg_from_dev(test_df, dev_df):
    # 将 dev_df 中的 neg_items 合并到 test_df 中
    dev_neg_dict = dev_df.set_index('user_id')['neg_items'].to_dict()  # user_id -> neg_items 映射
    
    neg_items_list = []
    for uid in test_df['user_id']:
        if uid in dev_neg_dict:  # 如果 dev 中有对应的负样本
            neg_items = dev_neg_dict[uid]
        # else:  # 如果 dev 中没有负样本，则随机生成
        #     n_items = test_df['item_id'].nunique()
        #     neg_items = np.random.randint(1, n_items + 1, 10).tolist()
        neg_items_list.append(neg_items)
    
    test_df['neg_items'] = neg_items_list
    return test_df


In [129]:
import numpy as np
leave_df = out_df.groupby('user_id').head(1)
data_df = out_df.drop(leave_df.index)

[dev_df], data_df = generate_dev_test(data_df)
t_df=generate_test_neg_from_dev(test_df,dev_df)
train_df = pd.concat([leave_df, data_df]).sort_index()

len(train_df), len(dev_df)

(780270, 29858)

In [130]:
train_df.head()

,user_id,item_id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [131]:
dev_df.head()

,user_id,item_id,neg_items
126,0,126,"[2733, 21244, 30404, 32104, 20758, 14936, 1543..."
175,1,175,"[37074, 38300, 23311, 31786, 15042, 23307, 369..."
259,2,258,"[22451, 15213, 9988, 16584, 34216, 5182, 11043..."
280,3,276,"[28022, 5411, 16822, 3899, 28588, 18232, 9725,..."
296,4,289,"[25275, 16677, 31076, 5347, 23418, 37545, 3116..."


In [ ]:
# save results

train_df.to_csv('train.csv', sep='\t', index=False)
dev_df.to_csv('dev.csv', sep='\t', index=False)
t_df.to_csv('test.csv', sep='\t', index=False)